In [1]:
import re

class McPNeuron:
    """A simple McCulloch-Pitts neuron.
    
    Object variables:
    name: a string, the name of the neuron
    theta: a non-negative integer, the threshold of the neuron
    x: a list of 0s and 1s, the excitatory inputs
    y: a list of 0s and 1s, the inhibitory inputs
    outputs: a list of 0s and 1s, the outputs of the neuron at time t=0,1,2,...
    
    Methods:
    reset: resets the neuron to its initial state
    update: calculates and adds the output to the outputs list of the neuron based on the inputs
    """
    
    def __init__(self, name, theta):
        """Initializes the neuron with a name and a threshold value."""
        self.name = name
        self.theta = theta
        self.reset()
        
        
    def reset(self):
        """Resets the neuron to its initial state."""
        self.x = []
        self.y = []
        self.outputs = [0]
        
        
    def update(self, trace=False):
        """Calculates and adds the output to the outputs list of the neuron based on the inputs."""
        if 1 in self.y:  # check if we have an inhibitory input -> 0
            if trace:
                print("Neuron {} inhibited (x={}, y={})".format(self.name, self.x, self.y))
            self.outputs.append(0)
        else:
            # calculate the sum of the inputs
            sum = 0
            for i in self.x:
                sum += i

            if sum >= self.theta:  # compare to the threshold, if equal or larger -> 1
                if trace:
                    print("Neuron {} activated (x={}, y={}, theta={})".format(self.name, self.x, self.y, self.theta))
                self.outputs.append(1)
            else:
                if trace:
                    print("Neuron {} not activated (x={}, y={}, theta={})".format(self.name, self.x, self.y, self.theta))
                self.outputs.append(0)
        
        
class McPNet:
    """A simple McCulloch-Pitts network.
    
    Object variables:
    neurons: a dictionary of McPNeuron objects, the neurons in the network (key=name of the neuron)
    connections: a dictionary of lists of tuples, the connections between the neurons (key=name of the target neuron)
    t: a non-negative integer, the current time step of the network
    """
    
    def __init__(self):
        self.neurons = {}
        self.connections = {}
        self.t = 0
    
        
    def addNeuron(self, name, theta):
        self.neurons[name] = McPNeuron(name, theta)
    
        
    def addConnection(self, source, target, type="x"):       

        if not re.match(r"input[0-9]+", source) and source not in self.neurons:
            raise Exception("Neuron {} does not exist".format(source))
        if target not in self.neurons:
            raise Exception("Neuron {} does not exist".format(target))
        if type not in ['x', 'y']:
            raise Exception("Invalid connection type {}".format(type))
        
        if not target in self.connections:
            self.connections[target] = []
        self.connections[target].append((source, type))
    
    
    def reset(self):
        """Resets the network to its initial state."""
        self.t = 0
        for neuron in self.neurons.values():
            neuron.reset()
            
            
    def step(self, inpt, trace=False):
        """Executes one time step of the network with the given input."""

        for neuron in self.neurons.values():
            neuron.x = []  # reset inputs
            neuron.y = []  # reset inputs
            for c in self.connections[neuron.name]:  # c[0] is the source neuron name, c[1] is the connection type (x or y)
                m = re.match(r"input([0-9]+)", c[0])  # try to match the source name to pattern "input" plus some decimal numbers
                if m:  # we have an input
                    value = inpt[int(m.group(1))]  # get the input value at postion given by the decimal number
                else:  # we have a neuron as source
                    value = self.neurons[c[0]].outputs[self.t]  # get the output of the source neuron at time t
                    
                if c[1] == "x":
                    neuron.x.append(value)  # add the input value to the excitatory inputs
                elif c[1] == "y":
                    neuron.y.append(value)  # add the input value to the inhibitory inputs
                else:
                    raise Exception("Invalid connection type {}".format(c[1]))
            
            neuron.update(trace=trace) # update the neuron's output
        
        self.t += 1  # increase the time step
        
        
    def run(self, inpt, steps=100):
        """Runs the network for the given number of steps."""
        self.reset()
        for i in range(steps):
            self.step(inpt, trace=False)
        
        
    def printOutputs(self, neurons=None):
        """Prints the output of the neurons in the network. If neurons is given, only the output of the specified neurons is printed."""
        
        for name in self.neurons.keys():
            if not neurons or name in neurons:
                print("Neuron {} output: {}".format(name, self.neurons[name].outputs[-1]))
            
            
    

# McCulloch-Pitts Neuron and Network Example

The order of steps for the McCulloch-Pitts neuron and network is as follows:
1. Create a McPNet object.
2. Add neurons to the network with the addNeuron method.
3. Add connections between neurons or from input with the addConnection method.
4. Run the network with the run method.
5. Print the output of the neurons with the printOutputs method.

In [16]:
#this code has been implemented by Seyedalireza Yaghoubi on May 2024


#create Counter network 
net = McPNet()
net.addNeuron("A(Number of 0)", theta=0)
net.addNeuron("B(Number of 1)", theta=1)
net.addNeuron("C(Number of 2)", theta=2)
net.addNeuron("D(Number of 3)", theta=3)
#add inputs [a,b,c] to neuron A
net.addConnection("input0", "A(Number of 0)", "y")
net.addConnection("input1", "A(Number of 0)", "y")
net.addConnection("input2", "A(Number of 0)", "y")
#add inputs [a,b,c] to neuron B
net.addConnection("input0", "B(Number of 1)", "x")
net.addConnection("input1", "B(Number of 1)", "x")
net.addConnection("input2", "B(Number of 1)", "x")
#add inputs [a,b,c] to neuron C
net.addConnection("input0", "C(Number of 2)", "x")
net.addConnection("input1", "C(Number of 2)", "x")
net.addConnection("input2", "C(Number of 2)", "x")
#add inputs [a,b,c] to neuron D
net.addConnection("input0", "D(Number of 3)", "x")
net.addConnection("input1", "D(Number of 3)", "x")
net.addConnection("input2", "D(Number of 3)", "x")
#add inhibitory inputs 
#if sum of the input exceed treshold the output result 1 so we have to make a upper treshold to avoid wrong answers.
net.addConnection("D(Number of 3)","C(Number of 2)","y")
net.addConnection("D(Number of 3)","B(Number of 1)","y")
net.addConnection("C(Number of 2)","B(Number of 1)","y")




for inpt in ((0,0,0), (1,0,0), (0,1,0), (0,0,1), (1,1,0), (1,0,1), (0,1,1), (1,1,1)):
    print("Counter network, input {}".format(inpt))
    net.run(inpt)
    net.printOutputs()


Counter network, input (0, 0, 0)
Neuron A(Number of 0) output: 1
Neuron B(Number of 1) output: 0
Neuron C(Number of 2) output: 0
Neuron D(Number of 3) output: 0
Counter network, input (1, 0, 0)
Neuron A(Number of 0) output: 0
Neuron B(Number of 1) output: 1
Neuron C(Number of 2) output: 0
Neuron D(Number of 3) output: 0
Counter network, input (0, 1, 0)
Neuron A(Number of 0) output: 0
Neuron B(Number of 1) output: 1
Neuron C(Number of 2) output: 0
Neuron D(Number of 3) output: 0
Counter network, input (0, 0, 1)
Neuron A(Number of 0) output: 0
Neuron B(Number of 1) output: 1
Neuron C(Number of 2) output: 0
Neuron D(Number of 3) output: 0
Counter network, input (1, 1, 0)
Neuron A(Number of 0) output: 0
Neuron B(Number of 1) output: 0
Neuron C(Number of 2) output: 1
Neuron D(Number of 3) output: 0
Counter network, input (1, 0, 1)
Neuron A(Number of 0) output: 0
Neuron B(Number of 1) output: 0
Neuron C(Number of 2) output: 1
Neuron D(Number of 3) output: 0
Counter network, input (0, 1, 1)
N